In [1]:
# sat_map.ipynb
import requests
import folium
import geopandas as gpd
from shapely.geometry import Point
from SPARQLWrapper import SPARQLWrapper, JSON

# --- 1. Hämta SAT-punkter från Wikidata ---
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?item ?itemLabel ?coord ?image WHERE {
  ?item wdt:P6104 wd:Q134294510;  # SAT
        wdt:P625 ?coord.
  OPTIONAL { ?item wdt:P18 ?image. }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

wd_points = []
for r in results["results"]["bindings"]:
    coord = r["coord"]["value"].replace("Point(", "").replace(")", "").split(" ")
    lon, lat = map(float, coord)
    wd_points.append({
        "name": r["itemLabel"]["value"],
        "lat": lat,
        "lon": lon,
        "image": r.get("image", {}).get("value")
    })

gdf_wd = gpd.GeoDataFrame(
    wd_points, geometry=[Point(p["lon"], p["lat"]) for p in wd_points], crs="EPSG:4326"
)

# --- 2. Hämta OSM-data längs leden (toalett + dricksvatten) ---
# Exempel: Overpass API query
overpass_url = "https://overpass-api.de/api/interpreter"
query = """
[out:json][timeout:50];
(
  node["amenity"="toilets"](59.0,18.0,60.5,19.5);
  node["amenity"="drinking_water"](59.0,18.0,60.5,19.5);
);
out center;
"""
osm_data = requests.get(overpass_url, params={'data': query}).json()

osm_points = []
for e in osm_data["elements"]:
    osm_points.append({
        "name": e["tags"].get("amenity"),
        "lat": e["lat"],
        "lon": e["lon"],
    })

gdf_osm = gpd.GeoDataFrame(
    osm_points, geometry=[Point(p["lon"], p["lat"]) for p in osm_points], crs="EPSG:4326"
)

# --- 3. Exportera GeoJSON ---
gdf_wd.to_file("output/sat_points.geojson", driver="GeoJSON")
gdf_osm.to_file("output/sat_facilities.geojson", driver="GeoJSON")

# --- 4. Skapa interaktiv karta ---
m = folium.Map(location=[59.5, 18.8], zoom_start=8)

for _, row in gdf_wd.iterrows():
    folium.Marker(
        [row["lat"], row["lon"]],
        popup=row["name"],
        icon=folium.Icon(color="green", icon="info-sign")
    ).add_to(m)

for _, row in gdf_osm.iterrows():
    folium.Marker(
        [row["lat"], row["lon"]],
        popup=row["name"],
        icon=folium.Icon(color="blue", icon="tint")
    ).add_to(m)

m.save("output/sat_map.html")
print("✅ Export klar: sat_points.geojson, sat_facilities.geojson, sat_map.html")


✅ Export klar: sat_points.geojson, sat_facilities.geojson, sat_map.html
